<a href="https://colab.research.google.com/gist/jxoesneon/fc7ce61dd77d53d129304390d434a5bf/zegion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import minimize

class Node:
    """
    Represents a node in a neural network.

    Attributes:
        parent (Node): The parent node in the network structure.
        siblings (list): List of sibling nodes (nodes in the same layer).
        num_inputs (int): Number of input connections to the node.
        weights (list): List of weights for each input connection.
        bias (float): Bias value of the node.
        output (float): Calculated output of the node.
        input_values (list): List of input values received by the node.
        learnable_parameter (float): A learnable parameter for activation functions like PReLU.
        alpha (float): Parameter for activation functions like ELU.
        activation_functions (list): List of available activation function names.
        activation_ranges (list): Ranges of selector values for each activation function.
        optimizer_functions (list): List of available optimizer names.
        optimizer_ranges (list): Ranges of selector values for each optimizer.
        timestep (int): Current timestep during training.
        previous_updates (list): Previous weight updates for momentum optimizer.
        squared_gradients (list): Accumulated squared gradients for Adagrad, RMSprop, Adadelta.
        squared_updates (list): Accumulated squared updates for Adadelta.
        first_moments (list): First moments (mean) for Adam, Nadam, AMSGrad.
        second_moments (list): Second moments (variance) for Adam, Nadam, AMSGrad.
        max_second_moments (list): Maximum of past second moments for AMSGrad.
        slow_weights (list): Slow weights for Lookahead optimizer.
        lbfgs_state (object): State information for L-BFGS optimizer.
    """
    def __init__(self, inputs, parent=None): # Pass the inputs list directly
        """
        Initializes a Node with random weights and bias.

        Args:
            inputs (list): Initial input values for the node. If fewer than 3 inputs
                           are provided, the list is extended with random values.
            parent (Node, optional): The parent node in the network structure.
                                     Defaults to None.
        """

        self.num_inputs = len(inputs) if len(inputs) > 3 else inputs.extend([np.random.uniform(0, 1) for n in range(3 - len(inputs))])  # Set num_inputs directly
        self.weights = [np.random.uniform(-1, 1) for _ in range(self.num_inputs)]
        self.bias = np.random.uniform(-1, 1)
        self.bias = np.random.uniform(-1, 1)
        self.output = 0
        self.input_values = []
        self.learnable_parameter = np.random.uniform(0, 1)
        self.alpha = np.random.uniform(0.1, 2.0)
        self.activation_functions = ["sigmoid", "tanh", "relu", "lrelu", "prelu", "softmax", "swish", "elu", "selu", "gelu"]
        self.activation_ranges = [(i/len(self.activation_functions), (i+1)/len(self.activation_functions)) for i in range(len(self.activation_functions))]
        self.optimizer_functions = ["gd", "momentum", "adagrad", "rmsprop", "adam", "adadelta", "nadam", "amsgrad", "lookahead", "lbfgs"]
        self.optimizer_ranges = [(i/len(self.optimizer_functions), (i+1)/len(self.optimizer_functions)) for i in range(len(self.optimizer_functions))]
        self.timestep = 1

        # Initialize optimizer-specific parameters
        self.initialize_optimizer_parameters()

    def initialize_optimizer_parameters(self):
        """Initializes parameters for all supported optimization algorithms."""
        self.previous_updates = [0] * len(self.weights)  # For momentum
        self.squared_gradients = [0] * len(self.weights)  # For Adagrad, RMSprop, Adadelta
        self.squared_updates = [0] * len(self.weights)  # For Adadelta
        self.first_moments = [0] * len(self.weights)  # For Adam, Nadam, AMSGrad
        self.second_moments = [0] * len(self.weights)  # For Adam, Nadam, AMSGrad
        self.max_second_moments = [0] * len(self.weights)  # For AMSGrad
        self.slow_weights = [w for w in self.weights]  # For Lookahead
        self.lbfgs_state = None  # For L-BFGS

    def calculate_output(self, weights):
        """
        Calculates the node's output given a set of weights.

        Args:
            weights (list): Weights to use for the calculation.

        Returns:
            float: The calculated output.
        """
        weighted_sum = sum(w * x for w, x in zip(weights, self.input_values[:-2])) + self.bias
        output = self.apply_activation(weighted_sum)
        return output

    def activate(self, inputs):
        """
        Calculates the output of the node based on input values.

        Args:
            inputs (list): Input values to the node, including activation
                           and optimizer selectors.

        Returns:
            float: The calculated output.

        Raises:
            ValueError: If the number of inputs is incorrect.
        """
        if len(inputs) != self.num_inputs:
            raise ValueError("Incorrect number of inputs.")
        self.input_values = inputs
        activation_selector = inputs[-1]  # Assuming the last input is the selector

        # Determine activation function based on selector value
        for i, (lower, upper) in enumerate(self.activation_ranges):
            if lower <= activation_selector < upper:
                self.activation_function = self.activation_functions[i]
                break
        else:  # If no range matches, default to the first activation function
            self.activation_function = self.activation_functions[0]

        self.output = self.calculate_output(self.weights)
        return self.output


    def apply_activation(self, weighted_sum, function_type="sigmoid",learnable_parameter=1):
        """
        Applies the selected activation function to the weighted sum.

        Args:
            weighted_sum (float): Weighted sum of inputs.
            function_type (str, optional): Name of the activation function to apply.
                                           Defaults to "sigmoid".
            learnable_parameter (float, optional): Learnable parameter for
                                                    activation functions like PReLU.
                                                    Defaults to 1.

        Returns:
            float: The result of applying the activation function.
        """
        # Choose your activation function
        match function_type:
          case "sigmoid":
            return 1 / (1 + np.exp(-weighted_sum))
          case "tanh":
            return (np.exp(weighted_sum) - np.exp(-weighted_sum)) / (np.exp(weighted_sum) + np.exp(-weighted_sum))
          case "relu":
            return np.max(0, weighted_sum)
          case "lrelu":
            return np.max(0.01*weighted_sum, weighted_sum)
          case "prelu":
            return np.max(learnable_parameter*weighted_sum, weighted_sum)
          case "softmax":
            return weighted_sum / np.sum(np.exp([sibling.output for sibling in self.siblings] - np.max([sibling.output for sibling in self.siblings])))
          case "swish":
            return weighted_sum * (1 / (1 + np.exp(-weighted_sum)))
          case "elu":
            return weighted_sum if weighted_sum > 0 else self.alpha * (np.exp(weighted_sum) - 1)
          case "selu":
            scale = 1.0507009873554804934193349852946
            selu_alpha = 1.6732632423543772848170429916717
            return scale * (weighted_sum if weighted_sum > 0 else selu_alpha * (np.exp(weighted_sum) - 1))
          case "gelu":
            return 0.5 * weighted_sum * (1 + np.tanh(np.sqrt(2 / np.pi) * (weighted_sum + 0.044715 * weighted_sum**3)))

    def softmax_derivative(self, output_vector):
        """
        Calculates the Jacobian matrix for the softmax function.

        Args:
            output_vector (numpy.ndarray): The vector of softmax outputs.

        Returns:
            numpy.ndarray: The Jacobian matrix.
        """
        jacobian = np.diagflat(output_vector) - np.outer(output_vector, output_vector)
        return jacobian

    def activation_derivative(self, output):
        """
        Calculates the derivative of the currently selected activation function.

        Args:
            output (float): The output value of the activation function.

        Returns:
            float or numpy.ndarray: The derivative of the activation function.
                                    Returns a numpy array for softmax.
        """
        match self.activation_function:
            case "sigmoid":
                return output * (1 - output)
            case "tanh":
                return 1 - output**2
            case "relu":
                return 1 if output > 0 else 0
            case "lrelu":
                return 1 if output > 0 else 0.01
            case "prelu":
                return 1 if output > 0 else self.learnable_parameter
            case "softmax":
                output_vector = np.array([sibling.output for sibling in self.siblings])
                return self.softmax_derivative(output_vector)
            case "swish":
                sigmoid = 1 / (1 + np.exp(-output))
                return sigmoid * (1 + output * (1 - sigmoid))
            case "elu":
                return 1 if output > 0 else self.alpha * np.exp(output)
            case "selu":
                scale = 1.0507009873554804934193349852946
                selu_alpha = 1.6732632423543772848170429916717
                return scale if output > 0 else scale * selu_alpha * np.exp(output)
            case "gelu":
                return 0.5 * (1 + np.tanh(np.sqrt(2 / np.pi) * (output + 0.044715 * output**3))) + 0.5 * output * (1 - np.tanh(np.sqrt(2 / np.pi) * (output + 0.044715 * output**3))**2) * (np.sqrt(2 / np.pi) * (1 + 0.134145 * output**2))


    def calculate_error(self, next_layer_nodes, next_layer_errors):
        """
        Calculates the error signal for this node using backpropagation.

        Args:
            next_layer_nodes (list): List of nodes in the next layer.
            next_layer_errors (list): List of errors from the next layer.

        Returns:
            float or numpy.ndarray: The calculated error signal for this node.
        """
        if not next_layer_nodes:  # Output layer
            # Assuming next_layer_errors is the target vector for the output layer
            return self.activation_derivative(self.output) * (self.output - next_layer_errors)
        else:
            error = 0
            for next_node, next_error in zip(next_layer_nodes, next_layer_errors):
                error += next_node.weights[self.siblings.index(self)] * next_error
            # For softmax, error is a vector, so we need to sum over the next layer's errors
            if self.activation_function == "softmax":
                return np.sum(error * self.activation_derivative(self.output), axis=0)
            else:
                return error * self.activation_derivative(self.output)

    def update_weights(self, learning_rate, next_layer_errors):
        """
        Updates the weights and bias of the node using the selected optimization algorithm.

        Args:
            learning_rate (float): The learning rate for the update.
            next_layer_errors (list): List of errors from the next layer.
        """
        optimizer_selector = self.input_values[-2]  # Assuming the second last input is the optimizer selector

        # Determine optimization function based on selector value
        for i, (lower, upper) in enumerate(self.optimizer_ranges):
            if lower <= optimizer_selector < upper:
                self.optimizer_function = self.optimizer_functions[i]
                break
        else:  # If no range matches, default to "gd"
            self.optimizer_function = "gd"

        error_term = self.calculate_error(self.siblings, next_layer_errors)

        self.apply_optimizer(learning_rate, error_term, recursion_depth=0)  # Call apply_optimizer with recursion depth

        self.bias -= learning_rate * error_term  # Update bias separately
        self.timestep += 1  # Increment timestep

    def apply_optimizer(self, learning_rate, error_term, recursion_depth=0):
        """
        Applies the selected optimization algorithm to update the weights.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term.
            recursion_depth (int, optional): Recursion depth for nested optimizers
                                            like Lookahead. Defaults to 0.
        """
        if recursion_depth >= 3:  # Limit recursion depth to 3
            self.apply_gd(learning_rate, error_term)  # Default to gradient descent
            return
        match self.optimizer_function:
            case "gd":
                self.apply_gd(learning_rate, error_term)
            case "momentum":
                self.apply_momentum(learning_rate, error_term)
            case "adagrad":
                self.apply_adagrad(learning_rate, error_term)
            case "rmsprop":
                self.apply_rmsprop(learning_rate, error_term)
            case "adam":
                self.apply_adam(learning_rate, error_term)
            case "adadelta":
                self.apply_adadelta(learning_rate, error_term)
            case "nadam":
                self.apply_nadam(learning_rate, error_term)
            case "amsgrad":
                self.apply_amsgrad(learning_rate, error_term)
            case "lookahead":
                self.apply_lookahead(learning_rate, error_term)
            case "lbfgs":
                self.apply_lbfgs(learning_rate, error_term)
            case _:  # Default to gradient descent
                self.apply_gd(learning_rate, error_term)

    def apply_gd(self, learning_rate, error_term):
        """
        Applies the Gradient Descent update rule to adjust the node's weights.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        for i, input_value in enumerate(self.input_values[:-2]):  # Exclude selector and activation inputs
            self.weights[i] -= learning_rate * error_term * input_value

    def apply_momentum(self, learning_rate, error_term):
        """
        Applies the Momentum update rule to adjust the node's weights,
        using the previous weight updates to accelerate gradient descent.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        if not hasattr(self, 'previous_updates'):
            self.previous_updates = [0] * len(self.weights)
        momentum_factor = 0.9
        for i, input_value in enumerate(self.input_values[:-2]):
            update = momentum_factor * self.previous_updates[i] - learning_rate * error_term * input_value
            self.weights[i] += update
            self.previous_updates[i] = update

    def apply_adagrad(self, learning_rate, error_term):
        """
        Applies the Adagrad update rule to adjust the node's weights.
        Adagrad adapts the learning rate for each weight based on the
        historical accumulation of squared gradients.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        if not hasattr(self, 'squared_gradients'):
            self.squared_gradients = [0] * len(self.weights)
        epsilon = 1e-8
        for i, input_value in enumerate(self.input_values[:-2]):
            self.squared_gradients[i] += (error_term * input_value)**2
            learning_rate_i = learning_rate / (np.sqrt(self.squared_gradients[i]) + epsilon)
            self.weights[i] -= learning_rate_i * error_term * input_value

    def apply_rmsprop(self, learning_rate, error_term):
        """
        Applies the RMSprop update rule to adjust the node's weights.
        RMSprop adapts the learning rate for each weight based on a
        moving average of squared gradients.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        if not hasattr(self, 'squared_gradients'):
          self.squared_gradients = [0] * len(self.weights)  # Initialize squared gradients to 0
          decay_rate = 0.9  # You can adjust this value
          epsilon = 1e-8  # Small value to prevent division by zero
          for i, input_value in enumerate(self.input_values[:-2]):  # Exclude selector and activation inputs
              self.squared_gradients[i] = decay_rate * self.squared_gradients[i] + (1 - decay_rate) * (error_term * input_value)**2
              learning_rate_i = learning_rate / (np.sqrt(self.squared_gradients[i]) + epsilon)
              self.weights[i] -= learning_rate_i * error_term * input_value

    def apply_adam(self, learning_rate, error_term):
        """
        Applies the Adam update rule to adjust the node's weights.
        Adam combines the benefits of Momentum and RMSprop, using
        moving averages of both gradients and squared gradients.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        if not hasattr(self, 'first_moments'):
            self.first_moments = [0] * len(self.weights)  # Initialize first moments (mean)
        if not hasattr(self, 'second_moments'):
            self.second_moments = [0] * len(self.weights)  # Initialize second moments (variance)
        beta1 = 0.9  # Exponential decay rate for first moment
        beta2 = 0.999  # Exponential decay rate for second moment
        epsilon = 1e-8  # Small value to prevent division by zero
        timestep = 1  # You'll need to increment this in your training loop

        for i, input_value in enumerate(self.input_values[:-2]):  # Exclude selector and activation inputs
            gradient = error_term * input_value
            self.first_moments[i] = beta1 * self.first_moments[i] + (1 - beta1) * gradient
            self.second_moments[i] = beta2 * self.second_moments[i] + (1 - beta2) * gradient**2

            # Bias correction
            first_moment_corrected = self.first_moments[i] / (1 - beta1**timestep)
            second_moment_corrected = self.second_moments[i] / (1 - beta2**timestep)

            self.weights[i] -= learning_rate * first_moment_corrected / (np.sqrt(second_moment_corrected) + epsilon)

    def apply_adadelta(self, learning_rate, error_term):
        """
        Applies the Adadelta update rule to adjust the node's weights.
        Adadelta adapts the learning rate for each weight based on a
        moving average of squared gradients and squared parameter updates.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        if not hasattr(self, 'squared_gradients'):
            self.squared_gradients = [0] * len(self.weights)  # Initialize squared gradients to 0
        if not hasattr(self, 'squared_updates'):
            self.squared_updates = [0] * len(self.weights)  # Initialize squared updates to 0
        decay_rate = 0.95  # You can adjust this value
        epsilon = 1e-8  # Small value to prevent division by zero

        for i, input_value in enumerate(self.input_values[:-2]):  # Exclude selector and activation inputs
            self.squared_gradients[i] = decay_rate * self.squared_gradients[i] + (1 - decay_rate) * (error_term * input_value)**2

            rms_gradient = np.sqrt(self.squared_gradients[i] + epsilon)
            rms_update = np.sqrt(self.squared_updates[i] + epsilon)

            update = - (rms_update / rms_gradient) * error_term * input_value
            self.weights[i] += update

            self.squared_updates[i] = decay_rate * self.squared_updates[i] + (1 - decay_rate) * update**2

    def apply_nadam(self, learning_rate, error_term):
        """
        Applies the Nadam update rule to adjust the node's weights.
        Nadam combines Adam with Nesterov momentum.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        if not hasattr(self, 'first_moments'):
            self.first_moments = [0] * len(self.weights)  # Initialize first moments (mean)
        if not hasattr(self, 'second_moments'):
            self.second_moments = [0] * len(self.weights)  # Initialize second moments (variance)
        if not hasattr(self, 'max_second_moments'):
            self.max_second_moments = [0] * len(self.weights)  # Initialize max second moments
        beta1 = 0.9  # Exponential decay rate for first moment
        beta2 = 0.999  # Exponential decay rate for second moment
        epsilon = 1e-8  # Small value to prevent division by zero
        timestep = 1  # You'll need to increment this in your training loop

        for i, input_value in enumerate(self.input_values[:-2]):  # Exclude selector and activation inputs
            gradient = error_term * input_value
            self.first_moments[i] = beta1 * self.first_moments[i] + (1 - beta1) * gradient
            self.second_moments[i] = beta2 * self.second_moments[i] + (1 - beta2) * gradient**2

            # Bias correction
            first_moment_corrected = self.first_moments[i] / (1 - beta1**timestep)
            second_moment_corrected = self.second_moments[i] / (1 - beta2**timestep)

            # Maintain maximum of past second moments
            self.max_second_moments[i] = np.maximum(self.max_second_moments[i], second_moment_corrected)

            self.weights[i] -= learning_rate * first_moment_corrected / (np.sqrt(self.max_second_moments[i]) + epsilon)

    def apply_amsgrad(self, learning_rate, error_term):
        """
        Applies the AMSGrad update rule to adjust the node's weights.
        AMSGrad is a variant of Adam that addresses potential issues
        with non-increasing learning rates.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
        """
        if not hasattr(self, 'first_moments'):
            self.first_moments = [0] * len(self.weights)
        if not hasattr(self, 'second_moments'):
            self.second_moments = [0] * len(self.weights)
        if not hasattr(self, 'max_second_moments'):
            self.max_second_moments = [0] * len(self.weights)
        beta1 = 0.9
        beta2 = 0.999
        epsilon = 1e-8

        for i, input_value in enumerate(self.input_values[:-2]):
            gradient = error_term * input_value
            self.first_moments[i] = beta1 * self.first_moments[i] + (1 - beta1) * gradient
            self.second_moments[i] = beta2 * self.second_moments[i] + (1 - beta2) * gradient**2

            first_moment_corrected = self.first_moments[i] / (1 - beta1**self.timestep)
            second_moment_corrected = self.second_moments[i] / (1 - beta2**self.timestep)

            self.max_second_moments[i] = np.maximum(self.max_second_moments[i], second_moment_corrected)

            self.weights[i] -= learning_rate * first_moment_corrected / (np.sqrt(self.max_second_moments[i]) + epsilon)

    def apply_lookahead(self, learning_rate, error_term, recursion_depth=0):
        """
        Applies the Lookahead update rule to adjust the node's weights.
        Lookahead maintains a set of slow weights that are updated
        periodically based on the updates made by an inner optimizer.

        Args:
            learning_rate (float): The learning rate for the update.
            error_term (float): The calculated error term for this node.
            recursion_depth (int, optional): Recursion depth for nested
                                            optimizers. Defaults to 0.
        """
        if not hasattr(self, 'slow_weights'):
            self.slow_weights = [w for w in self.weights]  # Initialize slow weights
        if not hasattr(self, 'inner_optimizer'):
            self.inner_optimizer = "adam"  # Choose inner optimizer
            # Initialize parameters for the inner optimizer
            self.initialize_optimizer_parameters()

        alpha = 0.5  # Slow weights update rate
        k = 5  # Number of inner optimizer steps before slow weights update

        # Inner optimizer step
        self.apply_optimizer(learning_rate, error_term, recursion_depth + 1)  # Increment recursion depth

        # Update slow weights every k steps
        if self.timestep % k == 0:
            for i in range(len(self.weights)):
                self.slow_weights[i] = self.slow_weights[i] + alpha * (self.weights[i] - self.slow_weights[i])
            self.weights = [w for w in self.slow_weights]  # Update weights with slow weights

    def apply_lbfgs(self, learning_rate, next_layer_errors):  # Include next_layer_errors as argument
        """
        Applies the L-BFGS update rule to adjust the node's weights.
        L-BFGS is a quasi-Newton optimization algorithm that approximates
        the inverse Hessian matrix to improve convergence.

        Args:
            learning_rate (float): The learning rate for the update.
            next_layer_errors (list): List of errors from the next layer.
        """
        if not hasattr(self, 'lbfgs_state'):
            self.lbfgs_state = None  # Initialize L-BFGS state

        def loss_function(weights, next_layer_errors=next_layer_errors):  # Capture next_layer_errors
            """Calculates the loss for the node given a set of weights."""
            output = self.calculate_output(weights)  # Call calculate_output

            # Use next_layer_errors as the target for this node
            target = next_layer_errors

            # Calculate the error based on your chosen loss function
            # Example with Mean Absolute Error (MAE):
            error = np.abs(output - target)
            return error

        # Use scipy.optimize.minimize with L-BFGS-B method (no changes here)
        result = minimize(loss_function, self.weights, method='L-BFGS-B', jac=True, options={'maxiter': 10})
        self.weights = list(result.x)  # Update node's weights with optimized values
        self.lbfgs_state = result.state  # Update L-BFGS state for next iteration



    def __str__(self):
        """
        Returns a string representation of the node for debugging and visualization.

        Returns:
            str: String representation of the node.
        """
        return f"Node with {self.num_inputs} inputs, weights: {self.weights}, bias: {self.bias}, output: {self.output}"

In [ ]:
nodelist = []
for _ in range(100):  # Create the desired number of nodes
    inputs = []  # Start with an empty list for each node
    inputs.extend([np.random.uniform(0, 1) for n in range(4)])  # Add 3 random inputs
    nodelist.append(Node(inputs))  # Create the Node and add it to the list
print(*nodelist, sep="\n")


Node with 4 inputs, weights: [-0.01678126546846581, 0.4389721629420553, -0.5841777848666678, -0.6743238851991002], bias: -0.5461425767418782, output: 0
Node with 4 inputs, weights: [0.6176861394256288, 0.23172006143279478, -0.46095706765309186, -0.206678997890956], bias: -0.789974371867481, output: 0
Node with 4 inputs, weights: [-0.5381100076164647, -0.7908718873792158, -0.197726644189393, -0.6979129118387657], bias: -0.9091854392450598, output: 0
Node with 4 inputs, weights: [-0.5595099134932371, 0.7751132429381633, -0.7693269493946966, 0.5267079314240901], bias: -0.8033921844567034, output: 0
Node with 4 inputs, weights: [-0.6982677627196949, 0.8471948713222615, 0.844037509759493, -0.2377393280379272], bias: -0.17411854921739223, output: 0
Node with 4 inputs, weights: [-0.2986774260104217, -0.19631560488676314, -0.14749163482223637, -0.06926839876301294], bias: 0.7364749653859919, output: 0
Node with 4 inputs, weights: [-0.2544192618467913, -0.2007776669366823, -0.2870382265893372, 

In [ ]:
import random

for node in nodelist:
    # Generate the correct number of inputs for each node + 2 selectors
    num_inputs_for_node = node.num_inputs
    nums = [random.randint(0, 10) for _ in range(num_inputs_for_node)]
    node.activate(nums)
print(*nodelist, sep="\n")

Node with 4 inputs, weights: [-0.01678126546846581, 0.4389721629420553, -0.5841777848666678, -0.6743238851991002], bias: -0.5461425767418782, output: 0.877634907046497
Node with 4 inputs, weights: [0.6176861394256288, 0.23172006143279478, -0.46095706765309186, -0.206678997890956], bias: -0.789974371867481, output: 0.9994317425553174
Node with 4 inputs, weights: [-0.5381100076164647, -0.7908718873792158, -0.197726644189393, -0.6979129118387657], bias: -0.9091854392450598, output: 0.004206841731634009
Node with 4 inputs, weights: [-0.5595099134932371, 0.7751132429381633, -0.7693269493946966, 0.5267079314240901], bias: -0.8033921844567034, output: 0.8849795173591528
Node with 4 inputs, weights: [-0.6982677627196949, 0.8471948713222615, 0.844037509759493, -0.2377393280379272], bias: -0.17411854921739223, output: 0.22680650569298696
Node with 4 inputs, weights: [-0.2986774260104217, -0.19631560488676314, -0.14749163482223637, -0.06926839876301294], bias: 0.7364749653859919, output: 0.177443

In [ ]:
class Layer:
    def __init__(self, num_nodes, num_inputs, activation_functions=None, optimizers=None,
                 regularization="none", normalization="none"):
        """
        Initializes a layer with flexible node configurations.

        Args:
          num_nodes: The number of nodes in the layer.
          num_inputs: The number of inputs each node receives.
          activation_functions: A list of activation functions for each node.
                                If None, defaults to "sigmoid" for all nodes.
          optimizers: A list of optimizers for each node.
                      If None, defaults to "gd" for all nodes.
          regularization: The type of regularization to apply (e.g., "l1", "l2", "dropout").
          normalization: The type of normalization to apply (e.g., "batch").
        """
        self.num_nodes = num_nodes
        self.nodes = []
        for i in range(num_nodes):
            activation_function = activation_functions[i] if activation_functions else "sigmoid"
            optimizer = optimizers[i] if optimizers else "gd"
            node = Node(num_inputs, activation_function, optimizer)
            self.nodes.append(node)
        self.regularization = regularization
        self.normalization = normalization
        # ... (add other parameters and initialization logic) ...

    def forward_pass(self, inputs):
        """
        Performs a forward pass through the layer, activating all nodes.

        Args:
          inputs: The input data to the layer.

        Returns:
          A list of outputs from all nodes in the layer.
        """
        outputs = []
        for node in self.nodes:
            outputs.append(node.activate(inputs))
        return outputs

    def update_weights(self, learning_rate, next_layer_errors):
        """
        Updates the weights of all nodes in the layer.

        Args:
          learning_rate: The learning rate for weight updates.
          next_layer_errors: A list of errors from the next layer.
        """
        for node in self.nodes:
            # ... (apply regularization before updating weights) ...
            node.update_weights(learning_rate, next_layer_errors)

    def __str__(self):

        """For debugging and visualization."""
        layer_str = ""
        for i, node in enumerate(self.nodes):
            layer_str += f"Node {i+1}: {node}\n"
        return layer_str


In [ ]:
input_data = [random.randint(0, 10) for _ in range(num_inputs_for_node)] # Example input data
layer1 = Layer(num_nodes=5, num_inputs=len(input_data))  # Create the first layer
layer2 = Layer(num_nodes=2, num_inputs=3, previous_layer=layer1)  # Create the second layer with layer1 as parent

layer1_outputs = layer1.activate(input_data)  # Activate the first layer
layer2_outputs = layer2.activate(layer1_outputs)  # Activate the second layer


TypeError: object of type 'int' has no len()

In [ ]:
class Network:
    def __init__(self, layer_sizes):  # List of layer sizes, e.g., [2, 3, 1]
        """Initializes a network with a given list of layer sizes."""
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            layer = Layer(layer_sizes[i + 1], layer_sizes[i])
            self.layers.append(layer)

    def activate(self, inputs):
        """Activates the entire network and returns the output of the last layer."""
        outputs = inputs
        for layer in self.layers:
            outputs = layer.activate(outputs)
        return outputs

    def update_weights(self, learning_rate, target_outputs):
        """Updates the weights of the entire network using backpropagation."""
        # Implement backpropagation here to calculate errors for each layer
        # ...
        # Then, update weights of each layer
        for layer in self.layers:
            # ... (pass the calculated errors to layer.update_weights) ...
            pass  # Replace with your backpropagation logic

    def __str__(self):
        """For debugging and visualization."""
        network_str = ""
        for i, layer in enumerate(self.layers):
            network_str += f"Layer {i+1}:\n{layer}\n"
        return network_str



In [ ]:
class Model:
    def __init__(self, network):
        """Initializes a model with a given network."""
        self.network = network

    def train(self, inputs, targets, learning_rate, epochs):
        """Trains the model on the given input and target data."""
        for epoch in range(epochs):
            for input_data, target_output in zip(inputs, targets):
                self.network.activate(input_data)
                self.network.update_weights(learning_rate, target_output)
            # ... (optional: calculate and print loss/metrics for the epoch) ...

    def predict(self, inputs):
        """Predicts outputs for the given inputs."""
        predictions = []
        for input_data in inputs:
            output = self.network.activate(input_data)
            predictions.append(output)
        return predictions

    def __str__(self):
        """For debugging and visualization."""
        return f"Model with network:\n{self.network}"



In [ ]:
# prompt: Scrape()  ## An Iterable to find Hi-Res videos on public archives

class Scrape:
    def __init__(self, url_list):
        self.url_list = url_list

    def __iter__(self):
        return self

    def __next__(self):
        if not self.url_list:
            raise StopIteration

        url = self.url_list.pop(0)
        # Here, you would implement the logic to scrape the URL and identify hi-res videos.
        # This may involve using libraries like requests, BeautifulSoup, or other scraping tools.
        # You would also need to specify the criteria for what constitutes a "hi-res" video.

        # Example (replace with actual scraping code):
        hi_res_video_url = self.simulate_scraping(url)

        if hi_res_video_url:
            return hi_res_video_url
        else:
            return self.__next__()  # Continue to the next URL if no hi-res video is found

    def simulate_scraping(self, url):
        # Replace this with actual scraping code
        if "high-res" in url:
            return url
        else:
            return None

# Example usage
url_list = ["https://archive.org/video1", "https://archive.org/video2", "https://archive.org/high-res-video3"]
scraper = Scrape(url_list)

for hi_res_video_url in scraper:
    print(f"Found hi-res video URL: {hi_res_video_url}")


In [ ]:
# prompt: Convert()  ## turns the video into multiple formats

!pip install ffmpeg-python

import ffmpeg
import os

def convert_video(input_file, output_dir):
  """Converts a video to multiple formats (MP4, WebM, Ogg)."""

  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  try:
    # Convert to MP4
    (
        ffmpeg
        .input(input_file)
        .output(os.path.join(output_dir, "output.mp4"), vcodec="libx264", acodec="aac")
        .run(overwrite_output=True)
    )
    print("Converted to MP4")

    # Convert to WebM
    (
        ffmpeg
        .input(input_file)
        .output(os.path.join(output_dir, "output.webm"), vcodec="libvpx", acodec="libvorbis")
        .run(overwrite_output=True)
    )
    print("Converted to WebM")

    # Convert to Ogg
    (
        ffmpeg
        .input(input_file)
        .output(os.path.join(output_dir, "output.ogg"), vcodec="libtheora", acodec="libvorbis")
        .run(overwrite_output=True)
    )
    print("Converted to Ogg")

  except ffmpeg.Error as e:
    print(f"Error converting video: {e}")


# Example usage:
# Replace 'input_video.mp4' with the actual path to your video file
# Replace 'output_folder' with the desired output directory
#convert_video('input_video.mp4', 'output_folder')


In [ ]:
# prompt: Downscale()  ## Downscales a video to multiple resolutions down to 480p using a multitude of downscaling algorithms

def downscale_video(input_file, output_dir):
    """Downscales a video to multiple resolutions down to 480p using various algorithms."""

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    resolutions = [720, 480]  # Add more resolutions as needed
    #downscaling_algorithms = ["lanczos", "bicubic", "bilinear"]
    downscaling_algorithms = ["lanczos"]


    try:
        for resolution in resolutions:
            for algorithm in downscaling_algorithms:
                output_file = os.path.join(output_dir, f"output_{resolution}p_{algorithm}.mp4")
                (
                    ffmpeg
                    .input(input_file)
                    .filter("scale", resolution, -2)
                    .output(output_file, vcodec="libx264", acodec="aac")
                    .run(overwrite_output=True)
                )
                print(f"Downscaled to {resolution}p with {algorithm} algorithm.")
    except ffmpeg.Error as e:
        print(f"Error downscaling video: {e}")


# Example usage
# Replace with the actual input video and output directory
# downscale_video('input_video.mp4', 'output_folder')


In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import time
class Scrape:
	def __init__(self, query, recursive=False, max_recursion=5):
		self.recursive = recursive
		self.max_recursion = max_recursion
		self.query = query
		self.results = []

	def google_search(self):
		url = f"https://www.google.com/search?q={self.query}&tbm=vid"
		headers = {
			"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.63 Safari/537.36"
		}
		response = requests.get(url, headers=headers)

		if response.status_code == 200:
			soup = BeautifulSoup(response.content, "html.parser")
			self.get_youtube_links(soup)

			if self.recursive:
				self.get_next_page(soup, headers)
		else:
			print(f"Failed to retrieve search results: {response.status_code}")

		return self.results

	def get_youtube_links(self, soup):
		link_tags = soup.find_all('a', href=True)
		for tag in link_tags:
			href = tag.get('href')
			if str(href.startswith("https://www.youtube.com/watch?v=")) in href or "youtu.be" in href:
				print(f'Found url: {href}')
				self.results.append(href)

	def get_next_page(self, soup, headers):
		next_button = soup.find('a', attrs={'id': 'pnnext'})
		if next_button and self.max_recursion > 0:
			self.max_recursion = self.max_recursion - 1
			next_url = 'https://www.google.com' + next_button.get('href')
			time.sleep(2)  # To avoid overwhelming the server
			next_response = requests.get(next_url, headers=headers)

			if next_response.status_code == 200:
				next_soup = BeautifulSoup(next_response.content, "html.parser")
				self.get_youtube_links(next_soup)
				self.get_next_page(next_soup, headers)  # Recursive call for the next page
			else:
				print(f"Failed to retrieve next page: {next_response.status_code}")

if __name__ == "__main__":
	query = "8k"
	scraper = Scrape(query, recursive=True)
	search_results = scraper.google_search()

	for r in search_results:
		print(r)
	print('Finished')


Finished


In [ ]:
import os
import ffmpeg
import signal
from concurrent.futures import ThreadPoolExecutor

class Scaler:
    def __init__(self, input_video, output_dir='video/output'):
        self.soft_stop = False  # Flag for soft stop
        self.immediate_stop = False  # Flag for immediate stop
        self.input_video = input_video
        self.output_dir = output_dir
        self.current_output_file = None  # To store the current output file being processed
        self.format_codecs = {
            "mp4": ["h264", "hevc", "mpeg4", "h264_videotoolbox"],
            "avi": ["mpeg4", "h263", "divx"],
            "mov": ["h264", "hevc", "mpeg4"],
            "mkv": ["vp9", "flac", "h264", "hevc"],
            "wmv": ["wmv2", "wmv3"],
            "flv": ["h264"],
            "webm": ["vp8", "vp9"],
            "mpeg": ["mpeg1", "mpeg2"],
            "3gp": ["h263", "h264"],
            "avchd": ["h264", "hevc"],
        }
        self.resolutions = [
            (7680, 4320),  # 8K
            (3840, 2160),  # 4K
            (2560, 1440),  # 1440p
            (1920, 1080),  # 1080p
            (1280, 720),   # 720p
            (854, 480),    # 480p
            (640, 360),    # 360p
        ]
        self.downscaling_options = {
            "sws_flags": [
                "fast_bilinear",
                "bilinear",
                "bicubic",
                "experimental",
                "neighbor",
                "area",
                "bicublin",
                "gauss",
                "sinc",
                "lanczos",
                "spline"
            ],
            "sws_dither": [
                "auto",
                "none",
                "bayer",
                "ed",
                "a_dither",
                "x_dither"
            ],
            "alphablend": [
                "uniform_color",
                "checkerboard",
                "none"
            ],
            "src_range": [0, 1],
            "dst_range": [0, 1],
            "param0": None,
            "param1": None
        }
        self.presets = ['ultrafast', 'superfast', 'veryfast', 'faster', 'fast', 'medium', 'slow', 'slower', 'veryslow']

        if not os.path.isfile(self.input_video):
            raise FileNotFoundError(f"The file {self.input_video} does not exist.")

        self.original_resolution = self.get_video_resolution()
        if self.original_resolution == (None, None):
            raise ValueError("Could not retrieve the original resolution, aborting conversion.")

        os.makedirs(self.output_dir, exist_ok=True)

    def signal_handler(self, sig, frame):
        """Handles the signals for graceful and immediate stops."""
        if sig == signal.SIGINT:  # Ctrl+C or Cmd+C
            print("\nSoft stop initiated. Waiting for current conversions to finish...")
            self.soft_stop = True
        elif sig == signal.SIGTERM:  # Ctrl+Shift+C or Cmd+Shift+C
            print("\nImmediate stop initiated. Deleting file in progress and exiting...")
            self.immediate_stop = True
            # Delete the file in progress (you'll need to track the current output file)
            if self.current_output_file:
                try:
                    os.remove(self.current_output_file)
                    print(f"Deleted {self.current_output_file}")
                except FileNotFoundError:
                    pass
            exit(1)

    def get_video_resolution(self):
        """Retrieve the resolution of the input video."""
        try:
            probe = ffmpeg.probe(self.input_video)
            video_stream = next(
                (stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None
            )
            if video_stream:
                width = int(video_stream['width'])
                height = int(video_stream['height'])
                return width, height
        except ffmpeg.Error as e:
            print(f"Error probing video resolution: {e.stderr.decode()}")
            return None, None

    def apply_resolution_scaling(self, stream, max_resolution, scaling_options=None):
        """Apply scaling to FFmpeg stream if video resolution exceeds max resolution."""
        current_width, current_height = self.original_resolution
        max_width, max_height = max_resolution

        if current_width > max_width or current_height > max_height:
            aspect_ratio = current_width / current_height
            if current_width / max_width > current_height / max_height:
                new_width = max_width
                new_height = int(max_width / aspect_ratio)
            else:
                new_height = max_height
                new_width = int(max_height * aspect_ratio)

            if scaling_options:
                scaling_flags = scaling_options.get("sws_flags", "bicubic")
                print(f"Scaling video to {new_width}x{new_height} using flags: {scaling_flags}")
                stream = stream.filter('scale', new_width, new_height, flags=scaling_flags)

                for option, value in scaling_options.items():
                    if option != "sws_flags" and value is not None:
                        stream = stream.filter('scale', **{option: value})
            else:
                print(f"Scaling video to {new_width}x{new_height}")
                stream = stream.filter('scale', new_width, new_height)

        return stream

    def convert_video_format(self, output_video, codec, max_resolution, scaling_options=None, preset='medium', crf=23, bv=None):
        """Convert to a single video format with specific FFmpeg arguments."""
        self.current_output_file = output_video  # Track the current output file
        if os.path.exists(output_video):
            if os.path.getsize(output_video) == 0:
                print(f"{output_video} exists but is zero-sized, removing and reconverting.")
                os.remove(output_video)
            else:
                print(f"Skipping {output_video}, file already exists and is non-zero.")
                return

        try:
            stream = ffmpeg.input(self.input_video)  # Create a NEW stream here
            stream = self.apply_resolution_scaling(stream, max_resolution, scaling_options)

            if bv:
                stream = stream.output(output_video, vcodec=codec, preset=preset, **{'b:v': bv})
            else:
                stream = stream.output(output_video, vcodec=codec, preset=preset, crf=crf)

            stream.run(capture_stdout=True, capture_stderr=True)
            print(f"Converted to {output_video}")

            if os.path.exists(output_video) and os.path.getsize(output_video) == 0:
                os.remove(output_video)
                print(f"Removed {output_video}, file size was zero.")

        except ffmpeg.Error as e:
            print(f"Error converting to {output_video}: {e.stderr.decode()}")

    def convert_to_all_formats(self):
        """Convert to all formats, codecs, resolutions, and downscaling options."""
        # Register signal handlers
        signal.signal(signal.SIGINT, self.signal_handler)
        signal.signal(signal.SIGTERM, self.signal_handler)

        self.soft_stop = False  # Flag for soft stop
        self.immediate_stop = False  # Flag for immediate stop
        self.current_output_file = None  # To store the current output file being processed

        base_name = os.path.splitext(os.path.basename(self.input_video))[0]

        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            for fmt, codecs in self.format_codecs.items():
                for codec in codecs:
                    for resolution in self.resolutions:
                        if resolution[0] > self.original_resolution[0] or resolution[1] > self.original_resolution[1]:
                            print(f"Skipping resolution {resolution[0]}x{resolution[1]} for format {fmt} with codec {codec} - exceeds original resolution.")
                            continue

                        for scaling_option in self.downscaling_options["sws_flags"]:
                            for preset in self.presets:
                                if self.soft_stop:  # Check for soft stop HERE
                                    print("Soft stop: Exiting loops.")
                                    return

                                options = self.downscaling_options.copy()
                                options["sws_flags"] = scaling_option

                                output_video_crf = os.path.join(self.output_dir, f"{base_name}_{fmt}_{codec}_{resolution[0]}x{resolution[1]}_{scaling_option}_{preset}_crf.{fmt}")
                                output_video_vbr = os.path.join(self.output_dir, f"{base_name}_{fmt}_{codec}_{resolution[0]}x{resolution[1]}_{scaling_option}_{preset}_vbr.{fmt}")

                                # Submit tasks ONLY if not soft stopping
                                executor.submit(self.convert_video_format, output_video_crf, codec, resolution, options, preset, crf=23)
                                executor.submit(self.convert_video_format, output_video_vbr, codec, resolution, options, preset, bv='2M')

if __name__ == "__main__":
    video_path = "video/input/video.mp4"
    output_dir = "video/output"
    scaler = Scaler(video_path, output_dir)
    scaler.convert_to_all_formats()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Skipping resolution 7680x4320 for format mp4 with codec h264 - exceeds original resolution.
Converted to video/output/video_mp4_h264_3840x2160_bilinear.mp4
Converted to video/output/video_mp4_h264_3840x2160_nearest.mp4
Converted to video/output/video_mp4_h264_3840x2160_bicubic.mp4
Converted to video/output/video_mp4_h264_3840x2160_fast_bilinear.mp4
Converted to video/output/video_mp4_h264_3840x2160_fast_bicubic.mp4
Converted to video/output/video_mp4_h264_3840x2160_lanczos.mp4
Converted to video/output/video_mp4_h264_3840x2160_spline.mp4
Converted to video/output/video_mp4_h264_3840x2160_box.mp4
Scaling video to 2560x1440 using nearest scaling.
Error converting to video/output/video_mp4_h264_2560x1440_nearest.mp4: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c

KeyboardInterrupt: 

In [ ]:
import shutil
from google.colab import files

# Define the folder you want to download
folder_to_save = '/content/video/output'

# Define the name of the zip file
zip_file = '/content/video_output.zip'

# Compress the folder into a zip file
shutil.make_archive(zip_file.replace('.zip', ''), 'zip', folder_to_save)

# Download the zip file
files.download(zip_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>